In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lifelines
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import find_best_parametric_model
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV,KFold
pd.set_option('display.max_columns', None) #show all columns in the dataframe
import re



<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def label_ICU (row):
    if row['ICU_1'] == 1 :
        return 1
    if row['ICU_2'] - row['ICU_1'] == 1:
        return 2
    if row['ICU_3'] - row['ICU_2'] == 1:
        return 3
    if row['ICU_4'] - row['ICU_3'] == 1:
        return 4
    if row['ICU'] - row['ICU_4'] == 1:
        return 5
    return 5

In [3]:
df_new=pd.read_pickle('BinaryICUPredicData.pkl')
df_new_1=pd.read_pickle('BinaryICUPredicData0to2.pkl')
ddf=pd.merge(df_new_1, df_new[['ICU_1','ICU_2','ICU_3','ICU_4','ICU_5']], left_index=True, right_index=True)


df = ddf.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df['ICUS']=df.apply(lambda row: label_ICU (row), axis=1)
df['ICU_cat']=df['ICUS']
df.loc[df.ICU == 0,'ICU_cat'] = 0

feature_cols = df.drop(columns = ['ICU_1','ICU_2','ICU_3','ICU_4','ICU_5','ICU_cat','ICU','ICUS',"PATIENT_VISIT_IDENTIFIER_1"]).columns.values

In [4]:
X = df[feature_cols]
y = df[['ICUS','ICU']]

X_train, X_validation, y_train, y_validation = train_test_split(X,y,test_size = 0.3)
trains=X_train.join(y_train)
validations=X_validation.join(y_validation)

y_cat = df[['ICU_cat']]
catX_train, catX_validation, caty_train, caty_validation = train_test_split(X,y_cat,test_size = 0.3)


In [5]:
dfkfold=X.join(y)

# Set up the 5-fold cross-validation
kfold = KFold(n_splits=5)
kfscore=[]
# Fit the CoxPHFitter on each fold and evaluate the performance
for train, test in kfold.split(dfkfold):
    cph = CoxPHFitter(penalizer=0.01)
    cph.fit(dfkfold.iloc[train],duration_col = 'ICUS', event_col = 'ICU')
    kfscore.append(cph.score(dfkfold,scoring_method='concordance_index'))
Modelscore=np.mean(kfscore)

print(Modelscore)

0.7464174145881887


In [6]:


#trains=X_train.join(y_train)
#validations=X_validation.join(y_validation)
df=X.join(y)

# Fit initial Cox model with all features
model = lifelines.CoxPHFitter(penalizer=0.01)
kfold = KFold(n_splits=5)


# Use AIC to identify the best model with one feature
best_model = model
best_score = float("-inf")
for col in X.columns:
    kfscore=[]
    for train, test in kfold.split(df):
        temp_model = CoxPHFitter(penalizer=0.01)
        temp_df=df[[col]].join(y)
        temp_model.fit(temp_df.iloc[train],duration_col = 'ICUS', event_col = 'ICU')
        kfscore.append(temp_model.score(df,scoring_method='concordance_index'))
    temp_score=np.mean(kfscore)
    print(temp_model.summary.p.values)
    if temp_score > best_score:
        best_model = temp_model
        best_score = temp_score




[9.87324707e-08]
[0.09240237]
[0.00074648]
[0.42211582]
[0.00840819]
[0.00187186]
[0.00272663]
[0.10060795]
[0.0031716]
[0.01003386]
[0.03909696]
[0.16516402]
[0.54962458]
[0.0351369]
[0.00983499]
[0.01861979]
[0.51932477]
[0.97686294]
[0.26650051]
[0.22533577]
[0.24637215]
[0.58334929]
[0.1135586]
[0.93100974]
[0.0008132]
[0.0002517]
[0.00169424]
[0.04037895]
[1.14139558e-06]
[9.75079585e-07]
[4.72536303e-05]
[0.70140597]
[0.12370399]
[0.11436477]
[0.14118174]
[0.00486499]
[0.53782245]
[0.27376582]
[0.70930049]
[0.93293876]
[0.45996991]
[0.02432064]
[0.08808588]
[1.54135376e-08]


In [7]:

# Use concordance_index to add additional features to the best model
while True:
    added_feature = False
    for col in X.columns:
        if col not in list(best_model.summary.index):
            kfscore=[]
            for train, test in kfold.split(df):
                temp_model = CoxPHFitter(penalizer=0.01)
                temp_df=df[list(best_model.summary.index)+ [col]].join(y)
                temp_model.fit(temp_df.iloc[train],duration_col = 'ICUS', event_col = 'ICU')
                kfscore.append(temp_model.score(df,scoring_method='concordance_index'))
            temp_score=np.mean(kfscore)
            if temp_score > best_score:
                best_model = temp_model
                best_score = temp_score
                added_feature = True
    if not added_feature:
        break

In [8]:
best_model.print_summary()

<lifelines.CoxPHFitter: fitted with 282 total observations, 146 right-censored observations>
             duration col = 'ICUS'
                event col = 'ICU'
                penalizer = 0.01
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 282
number of events observed = 136
   partial log-likelihood = -680.61
         time fit was run = 2022-12-13 03:36:16 UTC

---
                                 coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                              
RESPIRATORY_RATE_MEAN_1          1.90       6.69       0.86             0.22             3.58                 1.24                35.92
AGE_ABOVE65_1                    0.70       2.00       0.77            -0.81             2.20                 0.45                 9.01
GENDER_1                        -0.23       0.79       0.19            -0.61             0.14                 0.54                 1.15
HTN_1                            0.51       1.67       0.26            -0.01             1.03                 0.99                 2.80
BLOODPRESSURE_DIASTOLIC_DIFF_1  -0.66       0.52       1.56            -3.71             2.40                 0.02                10.98
BLOODPRESSURE_DIASTOLIC_MAX_1   -1.75       0.17       2.04            -5.75             2.24                 0.00                 9.39
BLOODPRESSURE_DIASTOLIC_MEAN_1  -0.51       0.60       1.42            -3.29             2.26                 0.04                 9.59
BLOODPRESSURE_SISTOLIC_DIFF_1    1.56       4.74       1.27            -0.93             4.04                 0.40                56.85
BLOODPRESSURE_SISTOLIC_MAX_1     1.38       3.97       1.89            -2.33             5.09                 0.10               162.32
BLOODPRESSURE_SISTOLIC_MEAN_1   -0.05       0.95       1.35            -2.70             2.59                 0.07                13.32
DISEASEGROUPING1_1              -0.52       0.59       0.32            -1.14             0.10                 0.32                 1.11
DISEASEGROUPING2_1              -0.04       0.96       0.55            -1.13             1.05                 0.32                 2.86
DISEASEGROUPING3_1               0.05       1.06       0.33            -0.59             0.70                 0.56                 2.01
DISEASEGROUPING4_1               1.34       3.82       0.54             0.28             2.40                 1.32                11.04
IMMUNOCOMPROMISED_1             -0.31       0.74       0.25            -0.80             0.18                 0.45                 1.20
OXYGEN_SATURATION_DIFF_1        -4.18       0.02       6.81           -17.53             9.17                 0.00              9579.63
OXYGEN_SATURATION_MAX_1          0.24       1.27       2.67            -4.99             5.48                 0.01               238.68
OXYGEN_SATURATION_MEAN_1        -2.10       0.12       2.56            -7.12             2.92                 0.00                18.53
OXYGEN_SATURATION_MIN_1          1.61       4.99       6.87           -11.86            15.07                 0.00             3.52e+06
RESPIRATORY_RATE_MIN_1          -1.31       0.27       0.81            -2.90             0.29                 0.05                 1.33
TEMPERATURE_DIFF_1               0.31       1.36       1.65            -2.93             3.55                 0.05                34.66
TEMPERATURE_MAX_1               -0.18       0.83       1.79            -3.69             3.32                 0.02                27.70
TEMPERATURE_MEAN_1               0.48       1.61       1.44            -2.35             3.30                 0.10                27.09
AGE_PERCENTIL_120th              0.18       1.20       0.48            -0.75             1.11                 0.47                 3.04
AGE_PERCENTIL_130th              0.76       

In [9]:
best_score

0.7499132245748005